<a href="https://colab.research.google.com/github/hdjcool/aiffel-first-repository/blob/master/%EC%BC%80%EC%B0%BD%EB%94%A509_%EB%AF%B8%EC%85%983.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers

In [ ]:
from tensorflow.keras.layers import RandomFlip, RandomRotation, RandomZoom # Import data augmentation layers


# Define data_augmentation as a Sequential model
data_augmentation = keras.Sequential(
    [
        RandomFlip("horizontal"), # Randomly flip images horizontally
        RandomRotation(0.1), # Randomly rotate images by up to 10%
        RandomZoom(0.1), # Randomly zoom in/out by up to 10%
    ]
)

In [ ]:
inputs = keras.Input(shape=(180,180,3))  # 입력 텐서 생성 - 입력 이미지의 공간적 크기는 180x180, 채널 개수는 3 (RGB)입니다.
x = data_augmentation(inputs)  # 데이터 증강 - 학습 데이터에 다양성을 추가하기 위한 전처리 과정입니다.
x = layers.Rescaling(1./255)(x)  # 입력 값의 정규화 - 입력 이미지를 [0, 1] 범위로 스케일링합니다.
x = layers.Conv2D(filters=32, kernel_size=5, use_bias=False)(x)  # 첫 번째 합성곱 층 - 필터 개수는 32개, 커널 크기는 5x5입니다.

for size in [32, 64, 128, 256, 512]:
    residual = x  # 잔차 연결용 변수 저장

    x = layers.BatchNormalization()(x)  # 배치 정규화 층 - 네트워크 안정성과 수렴 속도를 높이기 위해 사용합니다.
    x = layers.Activation("relu")(x)  # 활성화 함수 적용 - ReLU 활성화 함수입니다.
    x = layers.SeparableConv2D(size, 3, padding="same", use_bias=False)(x)  # 분리 합성곱 - 깊고 좁은 아키텍처에서 효율성을 높이기 위해 사용합니다.

    x = layers.BatchNormalization()(x)  # 배치 정규화 층 - 두 번째 배치 정규화 층입니다.
    x = layers.Activation("relu")(x)  # 활성화 함수 적용 - 두 번째 ReLU 활성화 함수입니다.
    x = layers.SeparableConv2D(size, 3, padding="same", use_bias=False)(x)  # 분리 합성곱 - 반복적인 분리 합성곱을 통해 표현력을 강화합니다.

    x = layers.MaxPooling2D(3, strides=2, padding="same")(x)  # 최대 풀링 층 - 공간적 크기를 줄이면서 주요 특징을 추출합니다.

    residual = layers.Conv2D(size, 1, strides=2, padding="same", use_bias=False)(residual)  # 잔차 연결 - 입력 크기를 출력 크기에 맞추기 위해 1x1 합성곱 사용.
    x = layers.add([x, residual])  # 잔차 연결 - 블록의 입력을 출력에 더해 네트워크의 학습을 용이하게 합니다.

x = layers.GlobalAveragePooling2D()(x)  # 전역 평균 풀링 층 - 최종 특성 맵의 공간적 크기를 축소하여 하나의 벡터로 만듭니다.
x = layers.Dropout(0.5)(x)  # 드롭아웃 - 과적합을 방지하기 위해 일부 뉴런을 무작위로 비활성화합니다.
outputs = layers.Dense(1, activation="sigmoid")(x)  # 출력 층 - 이진 분류를 위해 시그모이드 활성화 함수를 사용합니다.
model = keras.Model(inputs=inputs, outputs=outputs)  # 모델 생성 - 입력과 출력을 연결하여 모델을 정의합니다.


In [ ]:
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1             │ (None, 180, 180, 3)    │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ sequential (Sequential)   │ (None, 180, 180, 3)    │              0 │ input_layer_1[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ rescaling (Rescaling)     │ (None, 180, 180, 3)    │              0 │ sequential[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d (Conv2D)           │ (None, 176, 176, 32)   │          2,400 │ rescaling[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization       │ (None, 176, 176, 32)   │            128 │ conv2d[0][0]           │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation (Activation)   │ (None, 176, 176, 32)   │              0 │ batch_normalization[0… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ separable_conv2d          │ (None, 176, 176, 32)   │          1,312 │ activation[0][0]       │
│ (SeparableConv2D)         │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_1     │ (None, 176, 176, 32)   │            128 │ separable_conv2d[0][0] │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation_1 (Activation) │ (None, 176, 176, 32)   │              0 │ batch_normalization_1… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ separable_conv2d_1        │ (None, 176, 176, 32)   │          1,312 │ activation_1[0][0]     │
│ (SeparableConv2D)         │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d             │ (None, 88, 88, 32)     │              0 │ separable_conv2d_1[0]… │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_1 (Conv2D)         │ (None, 88, 88, 32)     │          1,024 │ conv2d[0][0]           │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ add (Add)                 │ (None, 88, 88, 32)     │              0 │ max_pooling2d[0][0],   │
│                           │                        │                │ conv2d_1[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_2     │ (None, 88, 88, 32)     │            128 │ add[0][0]              │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation_2 (Activation) │ (None, 88, 88, 32)     │              0 │ batch_normalization_2… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ separable_conv2d_2   

 Total params: 721,857 (2.75 MB)

 Trainable params: 718,849 (2.74 MB)

 Non-trainable params: 3,008 (11.75 KB)